In [1]:
import configparser
from pyspark.sql.types import *
from pyspark.sql.functions import * 

In [2]:
from etl import (create_spark_session, transform_load_location_dim)
from model import (create_model)
config = configparser.ConfigParser()
config.read("../dwh.cfg")
spark = create_spark_session(config.get("AWS", "KEY"), config.get("AWS", "SECRET"))

# Create analytics model

In [3]:
create_model(spark)
print("Succesfully created analytics model. Please configure and run etl.py")

Succesfully created analytics model. Please configure and run etl.py


# Data Dictionary

In [4]:
spark.sql("""describe date_dim """).show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|date_key|      int|   null|
|     day|  tinyint|   null|
|    week|  tinyint|   null|
|   month|  tinyint|   null|
| quarter|  tinyint|   null|
|    year|      int|   null|
+--------+---------+-------+



In [5]:
spark.sql("""describe trip_fact""").show(100, False)

+---------------------+-------------+-------+
|col_name             |data_type    |comment|
+---------------------+-------------+-------+
|vendor_key           |int          |null   |
|pickup_location_key  |int          |null   |
|drop_off_location_key|int          |null   |
|date_key             |int          |null   |
|passenger_count      |int          |null   |
|total_payment_amount |decimal(18,2)|null   |
|trip_count           |int          |null   |
+---------------------+-------------+-------+



In [6]:
spark.sql("""describe crash_dim""").show(100, False)

+-------------------------+-------------+-------+
|col_name                 |data_type    |comment|
+-------------------------+-------------+-------+
|date_key                 |int          |null   |
|location_key             |int          |null   |
|crash_count              |bigint       |null   |
|person_injury_count      |bigint       |null   |
|person_kill_count        |bigint       |null   |
|contributing_factor_list |array<string>|null   |
|impacted_participant_list|array<string>|null   |
|vehicle_type_list        |array<string>|null   |
+-------------------------+-------------+-------+



In [7]:
spark.sql("""describe location_dim""").show(100, False)

+------------+---------+-------+
|col_name    |data_type|comment|
+------------+---------+-------+
|location_key|int      |null   |
|zone        |string   |null   |
|borough     |string   |null   |
+------------+---------+-------+



In [8]:
spark.sql("""describe vendor_dim""").show(100, False)

+----------+---------+-------+
|col_name  |data_type|comment|
+----------+---------+-------+
|vendor_key|tinyint  |null   |
|name      |string   |null   |
+----------+---------+-------+



# Data Quality Checks

All target tables should have records > 0

In [3]:
    # Location dim ETL
df_taxi_zone_df = spark.read.option("header",True).csv(config.get("S3", "ZONE_LOOKUP_DATA_LOC"))
transform_load_location_dim(spark, df_taxi_zone_df)

In [19]:
spark.table("vendor_dim").count()

3

In [20]:
spark.table("trip_fact").count()

1243097

In [21]:
spark.table("location_dim").count()

265

In [17]:
spark.table("crash_dim").count()

225996

In [22]:
spark.table("date_dim").count()

10000

# Analytics

### Count all accidents involving taxi's


In [3]:
spark.sql("""
SELECT Count(*) 
FROM   crash_dim 
WHERE  Array_contains(vehicle_type_list, "TAXI") 
""").show()

+--------+
|count(1)|
+--------+
|   36041|
+--------+



### Identify vendors and crash_count, who's vechicle vechicle have likely been involved in the crash

In [14]:
spark.sql("""   
SELECT v.name, 
       Sum(c.crash_count) 
FROM   trip_fact t 
       JOIN vendor_dim v 
         ON t.vendor_key = v.vendor_key 
       JOIN date_dim d 
         ON d.date_key = t.date_key 
       JOIN location_dim l 
         ON t.pickup_location_key = l.location_key 
             OR t.drop_off_location_key = l.location_key 
       JOIN crash_dim c 
         ON c.date_key = d.date_key 
            AND c.location_key = l.location_key 
WHERE  Array_contains(c.vehicle_type_list, "TAXI") 
GROUP  BY v.name 
""").show(10, False)

+----------------------------+----------------+
|name                        |sum(crash_count)|
+----------------------------+----------------+
|Creative Mobile Technologies|116061          |
|VeriFone Inc.               |143804          |
|Not Available               |52886           |
+----------------------------+----------------+



### Find TOP 10 accident contributing factors involving taxis for each taxi vendor

In [15]:
spark.sql("""
SELECT   * 
FROM     ( 
                  SELECT   *, 
                           Rank() OVER (partition BY vendor ORDER BY factor_count DESC) AS rank 
                  FROM     ( 
                                    SELECT   vendor, 
                                             factor, 
                                             Count(*) AS factor_count 
                                    FROM     ( 
                                                    SELECT v.NAME                              AS vendor,
                                                           Explode(c.contributing_factor_list) AS factor
                                                    FROM   trip_fact t 
                                                    JOIN   vendor_dim v 
                                                    ON     t.vendor_key = v.vendor_key 
                                                    JOIN   date_dim d 
                                                    ON     d.date_key = t.date_key 
                                                    JOIN   location_dim l 
                                                    ON     t.pickup_location_key = l.location_key 
                                                    OR     t.drop_off_location_key = l.location_key
                                                    JOIN   crash_dim c 
                                                    ON     c.date_key = d.date_key 
                                                    AND    c.location_key = l.location_key 
                                                    WHERE  Array_contains(c.vehicle_type_list, "TAXI"))
                                    GROUP BY vendor, 
                                             factor)) 
WHERE    rank <= 10 
ORDER BY vendor, rank

""").show(100, False)

+----------------------------+------------------------------+------------+----+
|vendor                      |factor                        |factor_count|rank|
+----------------------------+------------------------------+------------+----+
|Creative Mobile Technologies|UNSPECIFIED                   |40790       |1   |
|Creative Mobile Technologies|DRIVER INATTENTION/DISTRACTION|25828       |2   |
|Creative Mobile Technologies|FOLLOWING TOO CLOSELY         |8727        |3   |
|Creative Mobile Technologies|PASSING OR LANE USAGE IMPROPER|7900        |4   |
|Creative Mobile Technologies|FAILURE TO YIELD RIGHT-OF-WAY |7044        |5   |
|Creative Mobile Technologies|OTHER VEHICULAR               |5026        |6   |
|Creative Mobile Technologies|PASSING TOO CLOSELY           |4956        |7   |
|Creative Mobile Technologies|UNSAFE LANE CHANGING          |4030        |8   |
|Creative Mobile Technologies|BACKING UNSAFELY              |3081        |9   |
|Creative Mobile Technologies|TURNING IM